# Data Exploration - Mac I
In the interest of not having to deal with merge conflicts (we later found a better solution), we elected to split our data exploration into two different files, one for each  of us!

## Importing everything

In [ ]:
% matplotlib inline
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection
from matplotlib import cm
from datetime import datetime
from ipywidgets import widgets  
from IPython.display import display

import seaborn as sns
print "imports imported"


Import crimedata and so some small transformations. 

Here, we're just transforming the data into an actual date-time object. 

In [ ]:
crimeData = pd.read_csv('train.csv')

crimeData['DateTime'] = crimeData['Dates'].apply(
    lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))

crimeData['Year'] = crimeData['DateTime'].apply(lambda x: x.year)
crimeData['Month'] = crimeData['DateTime'].apply(lambda x: x.month)
crimeData['Day'] = crimeData['DateTime'].apply(lambda x: x.day)
crimeData['Hour'] = crimeData['DateTime'].apply(lambda x: x.hour)

print "crimeData recoded"
crimeData

Generate a map of incidents for each type of crime

In [ ]:
mapdata = np.loadtxt("sf_map_copyright_openstreetmap_contributors.txt")
asp = mapdata.shape[0] * 1.0 / mapdata.shape[1]
clipsize = [[-122.5247, -122.3366],[ 37.699, 37.8299]]
lon_lat_box=[-122.52469, -122.33663, 37.69862, 37.82986]

crimeData['Xok'] = crimeData[crimeData.X<-121].X
crimeData['Yok'] = crimeData[crimeData.Y<40].Y

for cat in crimeData.Category.unique():

    crimeDataS = crimeData[crimeData.Category == cat]
    plt.figure()
    plt.grid(False)
    
    #ax = sns.kdeplot(crimeDataS.Xok, crimeDataS.Yok, clip=clipsize, aspect=1/asp)

    cmap = plt.get_cmap('gray')

    g = sns.regplot(x="Xok", y="Yok", data=crimeDataS, fit_reg=False, scatter_kws={'alpha':0.3})
    g.set_title(cat)
    g.imshow(mapdata, cmap=plt.get_cmap('gray'), 
                  extent=lon_lat_box, 
                  aspect=asp)

In the above graphs it is easy to see some clustering for certain crimes such as drugs and prostitution. This indicates that location will probably play a large role in predicting crimes. 

Function display a map of a given type of crime at a given hour

In [ ]:
def image_display(crime, time):
    #print crime,time
    crimeDataS = crimeData[crimeData.Category == crime][crimeData.Hour==time]
    plt.figure()
    plt.grid(False)
    
    #ax = sns.kdeplot(crimeDataS.Xok, crimeDataS.Yok, clip=clipsize, aspect=1/asp)

    cmap = plt.get_cmap('gray')

    g = sns.regplot(x="Xok", y="Yok", data=crimeDataS, fit_reg=False, scatter_kws={'alpha':0.3})
    g.set_title(cat)
    g.imshow(mapdata, cmap=plt.get_cmap('gray'), 
                  extent=lon_lat_box, 
                  aspect=asp)

Create Widgets to control map. By selecting the hour and crime type you can see a map of the crime locations. 

In [ ]:
vals = list(zip(crimeData.Category.unique(), crimeData.Category.unique()))
crime = widgets.Select(options=vals, description='Select one of the values:')
time = widgets.IntSlider(min=0, max=23, value=2003)
widgets.interact(image_display, crime = crime, time=time)

There is definitely some noticable trends in the time certain crimes occus and so it is quite likely this will also be helpful in predicting the type of crime that occured. 